# Q-learning

Y aunque no lo creas, aquí estamos, aprendiendo los algoritmos de **Aprendizaje por Reforzamiento**, y ya próximos a terminar de cubrir **Machine Learning** en estos 3 días tan provechosos.

Vamos entonces con **Q-Learning**, el mprimero de ellos


### ¿Y qué es Q-learning?

Q-learning es un tipo de algoritmo de **Aprendizaje por Refuerzo** que permite a un *agente* aprender a tomar decisiones óptimas y alcanzar un *objetivo* en un *ambiente* determinado.

El agente opera aprendiendo los valores más convenientes para cada paso que va a dar, y a esos pasos los implementamos en un par de datos que definen la **acción** y el **estado**.

Entonces en **Q-learning** tenemos un agente que debe identificar qué pasos realizar, y la calidad de esos pasos se denomina con la letra *"Q"*, por eso se llama *"Q-learning"*.

Los valores de *Q* (es decir, los valores de sus pasos, que se definen por los valores de **acción** y de **estado**) le ayudan al *agente* a decidir qué acción tomar en cada paso.

Imagínate que tu eres un **agente** que está parado en una **esquina X** de una **ciudad** que no conoces, y que quieres llegar a un **parque** que se encuentra en otro punto de la ciudad.

![ciudad](ciudad.png)

Ahora imagina que puedes tomar todas las **decisiones posibles** para moverte por las calles de la ciudad, y que en función de la acción tomada y el estado en que te encuentras al tomar cada decisión te permite evaluar qué tan buena ha sido esa decisión, en función de tu objetivo que es llegar al parque. Sin duda que después de probar **todas las alternativas posibles**, habrás hecho un aprendizaje sobre el camino correcto y sobre cuáles son las mejores decisiones posibles para llegar al parque soñado.

![ciudad](ciudad_pasos.png)



Para esto, lo que hace **Q-learning** detrás de la cortina, es actualizar los valores *Q* usando esta fórmula llamada la **ecuación de Bellman**.

![](Bellman.png)

No es necesario que la aprendas ni que la recuerdes. Solo te voy a describir muy superficialmente lo que hace para que luego, cuando escribamos código, entiendas para qué definimos ciertas variables.

Lo que dice esta ecuación es que el valor *Q* para un *estado* dado y una *acción* dada se actualiza tomando el valor *Q* original para ese *estado* y esa *acción*, y sumándole una fracción (definida por la *tasa de aprendizaje*) constituida por la **diferencia** entre:
+ La *recompensa* observada por tomar esa *acción* en ese *estado*, más el *valor máximo Q* estimado para el *siguiente estado* (que sería el *mejor valor Q* que podemos obtener desde el *siguiente estado*, considerando todas las posibles acciones futuras), multiplicado por el *factor de descuento* (que reduce el valor de las recompensas futuras),
+ Y todo eso restando el *valor Q original* para el *estado* y *acción* actuales.


### Basta de teoría

¿Te animas a implementarlo con un caso concreto? Seguro que sí: todos mis estudiantes son personas valientes. Especialmente los que han avanzado tanto en el curso.

Imaginemos un **robot** que opera en una **cuadrícula** que representa el salón de una fábrica en la que debe moverse trasladando herramientas. El objetivo es que el robot aprenda a encontrar el **camino más corto** desde su posición inicial hasta el **destino**, evitando **obstáculos**.

![](robot.png)

Entonces el entorno que vamos a definir será:
+ Una cuadrícula con dimensiones de 5x5.
+ El punto de inicio en la esquina superior izquierda (0,0).
+ El punto objetivo en la esquina inferior derecha (4,4).
+ Y los obstáculos distribuidos en la cuadrícula.

Vamos a establecer las **acciones**, que implicarán que el robot puede moverse en cuatro direcciones: *arriba*, *abajo*, *izquierda*, y *derecha*.

Y vamos a definir **recompensas**:
	• Alcanzar el objetivo: **+100**
	• Colisionar con un obstáculo: **-100**
	• Cualquier otro movimiento: **-1** (para incentivar la eficiencia)

Bueno, ha llegado el momento de implementarlo en Python, pero ten en cuenta que dada la complejidad de implementar un entorno de simulación desde cero, este ejemplo va a estar bastante simplificado para poder ilustrar el concepto en una lección de duración normal.

Siempre podrás ampliar estas habilidades con la documentación adecuada, pero yo me voy a asegurar de que comprendas estos conceptos para que luego estés en condiciones de crecer desde esta base.

Aunque parezca mentira, las únicas librerías que necesitaremos serán **NumPy** y **random**, para que nuestro agente pueda hacer cosas aleatorias.

In [1]:
import numpy as np
import random

El primer paso va a ser definir el entorno con sus **condiciones iniciales**. Esto sería básicamente como dibujar el tablero de juego.

In [2]:
dimensiones = (5, 5)
estado_inicial = (0, 0)
estado_objetivo = (4, 4)
obstaculos = [(1, 1), (1, 3), (2, 3), (3, 0)]
acciones = [(-1, 0), (1, 0), (0, -1), (0, 1)]

El siguiente paso es inicializar una nueva tabla, que se llama **tabla Q**, para la cual necesitamos definir dos variables auxiliares.
+ La primera es una variables que calcule el **número total de estados posibles** en el entorno. Es decir, según las dimensiones de la cuadrícula, cuántas posiciones posibles hay.

In [3]:
num_estados = dimensiones[0] * dimensiones[1]
num_estados

25

+ La segunda variable auxiliar que necesitará nuestra **tabla Q**, es una que nos diga **cuántas acciones diferentes** puede hacer nuestro robot en cada movida. En este caso sabemos que son cuatro, pero lo vamos a calcular de esta manera.

In [4]:
num_acciones = len(acciones)
num_acciones

4

Y finalmente definimos nuestra **tabla Q**, que va a ser una **matriz de dos dimensiones**.

In [5]:
Q = np.zeros((num_estados, num_acciones))
Q

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

Ahora que tenemos nuestra **tabla Q**, vamos a crear una **función** cuyo objetivo es convertir la representación bidimensional del *estado actual* (que sería la posición del robot en la cuadrícula) a un *índice lineal único*

¿Para qué hacemos esto? Para trabajar de manera más efectiva con la **tabla Q**, que es un array que contiene **25 filas** (cuyos índices van desde **0** hasta **24**), y de esa manera, podemos hacer que cada estado posible se pueda representar por un **número de índice** de la **tabla Q**.

In [6]:
def estado_a_indice(estado):
    return estado[0] * dimensiones[1] + estado[1]

Para que lo entendamos mejor apliquemos un ejemplo:

In [7]:
ejemplo = estado_a_indice((1, 0))
ejemplo

5

Puedes intentar jugar con algunos ejemplos más para identificar c+omo se realiza esta transformación.

El siguiente bloque de código va a definir los **parámetros clave** que se van a utilizar en el algoritmo de **Q-learning** dentro del contexto de nuestro ejemplo de navegación autónoma de un robot.

Esta puede ser la parte más abstracta de nuestro código, por lo que puede ser la más difícil de capturar, pero te la voy a explicar lo más claramente posible.

In [8]:
alpha = 0.1
gamma = 0.99
epsilon = 0.2
episodios = 100

+ **`alpha = 0.1`** - Este es el factor de **tasa de aprendizaje**. Controla cuánto se actualiza el **valor Q** en cada paso del aprendizaje. Un valor de alpha más alto significa que la información más reciente tiene un peso mayor, permitiendo un aprendizaje más rápido pero potencialmente menos estable. Un valor más bajo hace que el aprendizaje sea más lento pero puede llevar a una estimación más estable de los **valores Q**.
+ **`gamma = 0.99`** - Este es el **factor de descuento**. Lo que hace es determinar la importancia de las **recompensas futuras**. Si tenemos un valor de `gamma` cercano a `1` hace que las recompensas futuras sean casi tan importantes como las recompensas inmediatas, y de esa manera incentiva al agente a considerar consecuencias a largo plazo de sus acciones. Un valor más bajo haría que el agente valorase más las recompensas inmediatas.
+ **`epsilon = 0.2`** - Este valor sirve para que el agente **no repita siempre las mismas decisiones** ¿cómo? Definiendo la probabilidad de que el agente tome una acción aleatoria en lugar de la mejor acción conocida hasta el momento según la *tabla Q*. Esto permite que el agente explore el entorno en lugar de explotar constantemente el conocimiento que ya dispone. Con este número queremos lograr un equilibrio entre exploración y explotación para asegurar que el agente siga aprendiendo eficazmente sobre el entorno.
+ **`episodios = 1000`** - Aquí definimos el **número total de episodios** para el proceso de entrenamiento. Un **episodio** comienza con el agente en el estado inicial y termina cuando alcanza el objetivo o algún otro criterio de terminación. Mientras mayor sea el número de episodios, más oportunidades tendrá al agente de tener más experiencias de las cuales aprender, mejorando así potencialmente su política de acción.

Con estos parámetros, que son fundamentales, hemos configurado cómo el *agente* va a aprender a navegar por el *entorno*. Puedes ajustar estos valores luego para evaluar cómo afectan los resultados, teniendo en cuenta que esto puede tener un impacto muy significativo en la eficacia y en la eficiencia del aprendizaje del agente.

Ahora vamos a lo que a mí más me gusta, que es configurar funciones.

La primera, servirá para que el *agente* pueda elegir una **acción** de las 4 que tiene disponibles.

In [9]:
def elegir_accion(estado):
    if random.uniform(0, 1) < epsilon:
        return random.choice(range(num_acciones))
    else:
        return np.argmax(Q[estado_a_indice(estado)])

La segunda función servirá para **aplicar la acción elegida**.

In [10]:
def aplicar_accion(estado, accion_idx):
    accion = acciones[accion_idx]
    nuevo_estado = tuple(np.add(estado, accion) % dimensiones)
    
    if nuevo_estado in obstaculos or nuevo_estado == estado:
        return estado, -100, False
    if nuevo_estado == estado_objetivo:
        return nuevo_estado, 100, True
    return nuevo_estado, -1, False

Con todo esto, ya estamos en condiciones de iniciar el **proceso de entrenamiento**.

In [ ]:
for episodio in range(episodios):
    estado = estado_inicial
    terminado = False
    
    while not terminado:
        idx_estado = estado_a_indice(estado)
        accion_idx = elegir_accion(estado)
        nuevo_estado, recompensa, terminado = aplicar_accion(estado, accion_idx)
        idx_nuevo_estado = estado_a_indice(nuevo_estado)
        
        Q[idx_estado, accion_idx] = Q[idx_estado, accion_idx] + alpha * (recompensa + gamma * np.max(Q[idx_nuevo_estado]) - Q[idx_estado, accion_idx])
        
        estado = nuevo_estado

Muy bien, **nuestro modelo ha sido entrenado**, y ha definido una **política de acción** más conveniente para coada posición de nuestra cuadrícula.

Aquí podríamos decir que el trabajo está terminado, porque nuestro *agente* ya le puede transmitir al robot qué acciones tomar en cada posición para llegar hasta el objetivo siguiendo el camino más eficiente y sin chocar con ningún obstáculo.

Pero como nosotros además de lograr nuestros objetivos, nos gusta poder visualizar qué es lo que hemos hecho, vamos a diseñar una pequeña matriz que nos permita ver esto de un modo entendible.

### Todo listo. Ahora a visualizar

Crearemos una **matriz vacía** para la **política**, con las mismas dimensiones que el *entorno*.

In [12]:
politica = np.zeros(dimensiones, dtype=int)
politica

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

Y finalmente vamos a llenar la matriz de política con la **mejor acción** para cada *estado*.

In [13]:
for i in range(dimensiones[0]):
    for j in range(dimensiones[1]):
        estado = (i, j)
        idx_estado = estado_a_indice(estado)
        mejor_accion = np.argmax(Q[idx_estado])
        politica[i, j] = mejor_accion
        
print("Política aprendida (0: arriba, 1: abajo, 2: izquierda, 3: derecha)")
print(politica)

Política aprendida (0: arriba, 1: abajo, 2: izquierda, 3: derecha)
[[0 2 0 3 0]
 [0 0 1 0 1]
 [1 2 0 0 0]
 [0 2 2 2 0]
 [2 1 0 3 0]]


Este resultado es una instantánea del conocimiento adquirido por el *agente*: cómo navegar el entorno de manera eficiente basado en las recompensas y penalizaciones experimentadas durante el entrenamiento.

Y esto ha sido el **Q-learning**, el primero de los algoritmos de **Aprendizaje por Reforzamiento**. Te invito a aprender el segundo ahora mismo, en la siguiente lección.